In [ ]:
import pathlib
import pickle
import string
import sys
import re

import numpy as np
import pandas as pd

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
current_dir = pathlib.Path('.').resolve().parent
sys.path.append(str(current_dir))

In [ ]:
from src.dataset import Dataset
from src.trainer import Trainer
from src.models.convolution import ConvModel

In [ ]:
dataset = Dataset(vocabulary='../output/vocabulary.pkl',
                  tags='../output/tags.pkl',
                  dataset='../output/processed.sample.csv.gz')

In [ ]:
params = {'batch_size': 128,
          'shuffle': True,
          'num_workers': 4}

data_gen = torch.utils.data.DataLoader(dataset, **params)

In [ ]:
model = ConvModel(embedding_dim=32, vocab_size=len(dataset._embedder._vocabulary), seq_len=dataset.pad)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=0)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
trainer = Trainer(network=model, optimizer=optimizer, loss=loss_fn)

In [ ]:
trainer.train(input_loader=data_gen, n_epochs=2)